In [7]:
pip install pymongo langchain_openai # ou a lib do seu LLM preferido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.8 MB/s eta 0:00:00


In [8]:
import os
import pymongo
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import tiktoken # Necessário para obter tokens brutos do OpenAI

# --- Configuração da Chave da API (Exemplo com OpenAI) ---
# **ATENÇÃO:** Nunca coloque chaves de API diretamente no código em produção.
# Use variáveis de ambiente (recomendado) ou um arquivo .env.
# Ex: export OPENAI_API_KEY="SUA_CHAVE_AQUI" no terminal antes de rodar.
# Para este exemplo, estamos colocando diretamente para simplicidade, mas evite em produção.
os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_API_OPENAI_AQUI" # Substitua pela sua chave real

# --- Configuração do MongoDB ---
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "llm_tokens_db"
COLLECTION_NAME = "token_usage"

def store_token_usage_in_mongodb(text, token_count, llm_model_name, collection, raw_tokens=None):
    """
    Armazena os dados de tokenização no MongoDB.

    Args:
        text (str): O texto original que foi tokenizado.
        token_count (int): O número de tokens.
        llm_model_name (str): O nome do modelo LLM usado para tokenizar.
        collection (pymongo.collection.Collection): A coleção MongoDB para inserir os dados.
        raw_tokens (list, optional): A lista de tokens brutos (se disponível). Defaults to None.
    """
    try:
        document = {
            "original_text": text,
            "token_count": token_count,
            "llm_model": llm_model_name,
            "timestamp": pymongo.datetime.datetime.now(), # Adiciona um timestamp para rastreamento
        }
        if raw_tokens:
            document["raw_tokens"] = raw_tokens

        result = collection.insert_one(document)
        print(f"Dados de token inseridos no MongoDB com ID: {result.inserted_id}")

    except Exception as e:
        print(f"Erro ao armazenar dados no MongoDB: {e}")

if __name__ == "__main__":
    # Inicializar o modelo de LLM.
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    # Texto de exemplo para tokenização
    text_to_tokenize = "O Langchain é um framework poderoso para construir aplicações com modelos de linguagem grandes."

    # --- 1. Contar tokens com Langchain ---
    token_count = llm.get_num_tokens(text_to_tokenize)
    print(f"Número de tokens para o texto: {token_count}")

    # --- 2. Obter tokens brutos (específico para OpenAI/tiktoken) ---
    decoded_tokens = None
    try:
        encoding = tiktoken.encoding_for_model(llm.model_name)
        raw_token_ids = encoding.encode(text_to_tokenize)
        decoded_tokens = [encoding.decode_single_token_bytes(t).decode('utf-8', errors='ignore') for t in raw_token_ids]
        print(f"Tokens decodificados (tiktoken): {decoded_tokens}")
    except ImportError:
        print("tiktoken não está instalado. Não é possível obter tokens brutos para modelos OpenAI.")
    except Exception as e:
        print(f"Erro ao obter tokens brutos com tiktoken: {e}")

    # --- 3. Conectar ao MongoDB e Armazenar ---
    client = None # Inicializa client como None
    try:
        client = pymongo.MongoClient(MONGO_URI)
        db = client[DB_NAME]
        token_collection = db[COLLECTION_NAME]
        print("Conexão com MongoDB estabelecida com sucesso.")

        store_token_usage_in_mongodb(
            text=text_to_tokenize,
            token_count=token_count,
            llm_model_name=llm.model_name,
            collection=token_collection,
            raw_tokens=decoded_tokens # Opcional: passa os tokens brutos se disponíveis
        )

    except pymongo.errors.ConnectionFailure as e:
        print(f"Erro de conexão com o MongoDB: {e}")
    except Exception as e:
        print(f"Ocorreu um erro geral: {e}")
    finally:
        if client: # Verifica se o cliente foi inicializado antes de fechar
            client.close()
            print("Conexão com MongoDB fechada.")

Número de tokens para o texto: 21
Tokens decodificados (tiktoken): ['O', ' Lang', 'chain', ' é', ' um', ' framework', ' poder', 'oso', ' para', ' constr', 'uir', ' ap', 'lica', 'ções', ' com', ' modelos', ' de', ' lingu', 'agem', ' grandes', '.']
Conexão com MongoDB estabelecida com sucesso.
Erro ao armazenar dados no MongoDB: module 'pymongo' has no attribute 'datetime'
Conexão com MongoDB fechada.
